# Project Orange: IMDB reviews sentiment analysis

The purpose of the project is to build a model that would be able to classify the sentiment of a review of a movie and label if it has a positive or a negative sentiment. This way the opinions could be classified to be either negative or positive, while the medium sentiment is excluded from the analysis. 
Thus, the goal is to assign and weight either a positive or a negative connotation associated with each word or group of words. In addition, no weight is intended to be assigned to the words that are commonly used is sentence formation and that do not reflect any emotion. 

# 1. Data Cleaning

## Setup

Install all required dependencies for the future analysis in the current Jupyter kernel.

In [3]:
import sys
!{sys.executable} -m pip install -U scikit-learn spacy pandas seaborn sklearn nltk jupyter
!{sys.executable} -m spacy download en_core_web_sm

Requirement already up-to-date: scikit-learn in /Users/sylvain/anaconda3/envs/optimize/lib/python3.7/site-packages (0.22)
Requirement already up-to-date: spacy in /Users/sylvain/anaconda3/envs/optimize/lib/python3.7/site-packages (2.2.3)
Requirement already up-to-date: pandas in /Users/sylvain/anaconda3/envs/optimize/lib/python3.7/site-packages (0.25.3)
Requirement already up-to-date: seaborn in /Users/sylvain/anaconda3/envs/optimize/lib/python3.7/site-packages (0.9.0)
Requirement already up-to-date: sklearn in /Users/sylvain/anaconda3/envs/optimize/lib/python3.7/site-packages (0.0)
Requirement already up-to-date: nltk in /Users/sylvain/anaconda3/envs/optimize/lib/python3.7/site-packages (3.4.5)
Requirement already up-to-date: jupyter in /Users/sylvain/anaconda3/envs/optimize/lib/python3.7/site-packages (1.0.0)


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import spacy
import nltk.corpus

from nltk.corpus import words


from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, average_precision_score
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve


%matplotlib inline
sns.set_style("darkgrid")
sp = spacy.load('en_core_web_sm')

### Import dataset

In [5]:
data = pd.read_csv("../data/imdb_dataset.csv") 

# Keep the first 100 elements to reduce the load on cpu
data=data[:1000]
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Cleaning

#### Text to lowercase

In [6]:
def to_lower(this_review):
    this_review=this_review.lower()
    return this_review

#### Remove HTML elements

In [7]:
REMOVE_HTML = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def remove_html(review):
    return REMOVE_HTML.sub(" ", review) 

#### Identify and remove entities

In [8]:
def remove_entities(this_review):
    doc=sp(this_review)
    
    for i in doc.ents:
            i=str(i)
            this_review=this_review.replace(" "+i,"")
    return this_review

#### Lemmatization

In [9]:
# Implementing lemmatization
def lemmatize_it(this_review):
    filtered_sent=[]

    #  "nlp" Object is used to create documents with linguistic annotations.
    lem = sp(this_review)
    
   # finding lemma for each word
    for word in lem:
        filtered_sent.append(word.lemma_)
    return filtered_sent

#### Tokenization (not used)

In [10]:
# "nlp" Object is used to create documents with linguistic annotations.
nlp = spacy.lang.en.English()

def tokenize_review(this_review):
    my_doc = nlp(this_review)
    
    # Create list of word tokens
    token_list = []
    for token in my_doc:
        token_list.append(token.text)
    return token_list

#### Adapt spacy stopwords list to our topic

In [11]:
#print stopword list from spacy
spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)

remove_from_stopwordlist=["n't", "most", "much", "never", "no", "not", "nothing", "n‘t", "n’t", "really", "top", "very", "well"]
for word in spacy_stopwords:
    if word in remove_from_stopwordlist:
         spacy_stopwords.remove(word)

add_to_stopwords=['.', ',', '!', '?', ':', '&', '...', '(', ')','-', '/', '"', ';', '-PRON-', ' ', "'", '....', '  ', '*']
for word in add_to_stopwords:
    spacy_stopwords.append(word)

####  Remove stopwords and punctuation

In [12]:
def eliminate_stopwords(this_review):
    
    filtered_sent=[]

    #  "nlp" Object is used to create documents with linguistic annotations.
    doc = this_review
    
    # filtering stop words
    for word in doc:
        if word not in spacy_stopwords:
            filtered_sent.append(word)
    return filtered_sent
    

#### Check if tokens are words and remove those that aren't (using NLTK)

In [13]:
nltk.download('words')
def remove_non_words(this_review):
    clean_sent=[]
    for word in this_review:
        if word in words.words():
            clean_sent.append(word)
    return clean_sent        

[nltk_data] Downloading package words to /Users/sylvain/nltk_data...
[nltk_data]   Package words is already up-to-date!


#### reconcatenate list to string

In [14]:
def reconcatenate_list_to_string (this_review):
    this_review=' '.join(this_review)
    return this_review

## Final dataset

In [23]:
def cleaning(this_review):
    this_review=to_lower(this_review)
    this_review=remove_entities(this_review)
    this_review=lemmatize_it(this_review)
    this_review=eliminate_stopwords(this_review)
    this_review=remove_non_words(this_review)
    this_review=reconcatenate_list_to_string (this_review)
    
    return this_review

In [24]:
#data['cleaned_reviews'] = data['review'].map(cleaning)
#data.head()

In [25]:
#Process all reviews and save to csv
#data = pd.read_csv("../data/imdb_dataset.csv") 

#data['review'] = data['cleaned_reviews']
#data.to_csv("../data/clean_dataset.csv", index=False)
